# Evaluate the classifier's transferability on a tiled dataset from Bern cohorts

In [15]:
import os
import functools

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset
from torchvision.datasets import ImageFolder
from sklearn.metrics import f1_score, accuracy_score
import numpy as np
from tqdm import tqdm

import Abed_utils

## Load and build model

In [2]:
patch_size = 8
batch_size = 8
im_size = 224
path_to_backbone = './ckpts/dino_deitsmall8_pretrain.pth'
path_to_classifier = './ckpts/classifier_K19_CE_100ep_one_layer.pt'

transform = functools.partial(Abed_utils.normalize_input, im_size=im_size, patch_size=patch_size)

In [14]:
backbone = Abed_utils.get_vit(patch_size, path_to_backbone)

classifier = Abed_utils.ClassificationHead(pretrained_path=path_to_classifier)
# features, labels = Abed_utils.load_features(os.path.join(Abed_utils.OUTPUT_ROOT, 'features'), cuda=True)
# classifier = Abed_utils.KNNClassifier(features, labels)
model = nn.Sequential(backbone, classifier)
# model = classifier
model.eval()

Sequential(
  (0): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 384, kernel_size=(8, 8), stride=(8, 8))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0): Block(
        (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=384, out_features=1152, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=384, out_features=384, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (act): GELU()
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
      (1): Block(
        (norm1): LayerNorm((384,), eps=1e-

In [18]:
if Abed_utils.BERN_TILES_ROOT is None:
    raise RuntimeError('labelled Bern tiles not available on this machine, run on the dataserver')

wsis = os.listdir(Abed_utils.BERN_TILES_ROOT)
with torch.no_grad():
    for wsi in wsis:
        acc = 0
        f1 = 0
        ds = ImageFolder(os.path.join(Abed_utils.BERN_TILES_ROOT, wsi), transform=transform)
        # features, labels = Abed_utils.load_features(os.path.join(Abed_utils.OUTPUT_ROOT, f'features-{wsi}'), cuda=True)
        # ds = TensorDataset(features, labels)
        data = DataLoader(ds, batch_size=batch_size)
        preds = np.empty(len(ds))
        targets = np.empty_like(preds)
        for i, (x, y) in enumerate(tqdm(data)):
            idx = slice(i*batch_size,min((i+1)*batch_size, preds.shape[0]))
            predictions = model(x.cuda()).cpu().numpy()
            preds[idx] = predictions if len(predictions.shape) < 2 else predictions.argmax(axis=1)
            targets[idx] = y.cpu().numpy()

        acc = (preds == targets).mean()
        f1 = f1_score(targets, preds, average='micro')
        print(f'for {wsi} we get an accuracy of {acc:.4f} and an f1 of {f1:.4f}')


100%|██████████| 4006/4006 [20:07<00:00,  3.32it/s]


for 001b_B2005.30530_C_HE.mrxs we get an accuracy of 0.4220 and an f1 of 0.4220


100%|██████████| 3322/3322 [17:04<00:00,  3.24it/s]


for 352b_B2005.5775_F_HE.mrxs we get an accuracy of 0.0370 and an f1 of 0.0370


100%|██████████| 3362/3362 [17:23<00:00,  3.22it/s]

for 565c_B2012.15587_B_HE.mrxs we get an accuracy of 0.1799 and an f1 of 0.1799
